# An introduction to `relatio` 
**Runtime $\sim$ 60min**



----------------------------

This is a short demo of the package `relatio`.  It takes as input a text corpus and outputs a list of narrative statements. The pipeline is unsupervised: the user does not need to specify narratives beforehand. Narrative statements are defined as tuples of semantic roles with a (agent, verb, patient) structure. 

Here, we present the main functions to quickly obtain narrative statements from a corpus.

----------------------------

In this tutorial, we work with the Wealth of Nations.

----------------------------

In [ ]:
# Catch warnings for an easy ride
from relatio import FileLogger
logger = FileLogger(level = 'WARNING')

In [ ]:
import pandas as pd
import os

# Load local CSV dataset
data_path = "data/cleaned_data.csv"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)

In [ ]:
from relatio import Preprocessor

# Enhanced preprocessor for better clause handling
p = Preprocessor(
    spacy_model = "en_core_web_sm",
    remove_punctuation = False,  # Keep punctuation for clause boundary detection
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    remove_chars = ["\"","^","?","!","(",")",":","\'","+","&","|","/","{","}",
                    "~","_","`","[","]",">","<","=","*","%","$","@","#","'"],
    # Keep important punctuation for clause structure: . , ; -
    stop_words = [],
    n_process = -1,
    batch_size = 50  # Reduced for better processing of complex sentences
)

In [ ]:
df = p.split_into_sentences(
    df, output_path = None, progress_bar = True
)

In [ ]:
from relatio import SRL
import re
import warnings

warnings.filterwarnings('ignore')

print("Skipping SRL model - using alternative sentence processing")

# Check if we have data loaded
if 'df' not in locals():
    print("Error: DataFrame not loaded. Please run previous cells first.")
    filtered_roles = []
else:
    # Ensure we have the 'sentence' column
    if 'sentence' not in df.columns:
        if 'doc' in df.columns:
            df['sentence'] = df['doc']
            print("Renamed 'doc' column to 'sentence'")
        else:
            print(f"Available columns: {df.columns.tolist()}")
            # Create a dummy sentence column
            df['sentence'] = df.iloc[:, 1] if len(df.columns) > 1 else "sample text"
    
    print(f"Processing {len(df)} sentences from dataset")
    
    # 更全面的代词和指代词过滤集合
    pronouns_to_filter = {
        # 人称代词
        'i', 'me', 'my', 'mine', 'myself',
        'you', 'your', 'yours', 'yourself', 'yourselves',
        'he', 'him', 'his', 'himself',
        'she', 'her', 'hers', 'herself',
        'it', 'its', 'itself',
        'we', 'us', 'our', 'ours', 'ourselves',
        'they', 'them', 'their', 'theirs', 'themselves',
        
        # 指示代词
        'this', 'that', 'these', 'those',
        
        # 关系代词（重要！）
        'which', 'who', 'whom', 'whose', 'that',
        
        # 疑问代词
        'what', 'where', 'when', 'why', 'how',
        
        # 不定代词
        'one', 'ones', 'another', 'other', 'others',
        'some', 'any', 'all', 'both', 'each', 'either', 'neither',
        'few', 'many', 'several', 'most', 'much',
        'such', 'same', 'latter', 'former',
        
        # 复合代词
        'someone', 'somebody', 'something',
        'anyone', 'anybody', 'anything',
        'everyone', 'everybody', 'everything',
        'no one', 'nobody', 'nothing',
        'somewhere', 'anywhere', 'everywhere', 'nowhere'
    }
    
    # Create simple role structures for now (will be enhanced by later cells)
    print("Creating simplified role structures...")
    filtered_roles = []
    
    # Sample a few sentences to create basic role structures
    sample_size = min(100, len(df))
    for i in range(sample_size):
        try:
            sentence = str(df['sentence'].iloc[i])
            words = sentence.lower().split()
            
            # Skip very short sentences
            if len(words) < 5:
                continue
            
            # Create a basic role structure (this will be enhanced by SVO extraction later)
            role = {
                'sentence_id': i,
                'text': sentence,
                'length': len(words)
            }
            filtered_roles.append(role)
            
        except Exception as e:
            continue
    
    print(f"Created {len(filtered_roles)} basic role structures")
    print("Note: SRL model skipped - will use SVO extraction in next cell")
    print(f"Sample roles: {filtered_roles[:3] if filtered_roles else 'None'}")
    
    # Set global variable for use in next cells
    globals()['pronouns_to_filter'] = pronouns_to_filter

In [ ]:
# 导入spaCy用于指代消解
import spacy
import re
from collections import defaultdict

print("开始SVO提取和指代消解...")

# 加载spaCy模型
try:
    nlp = spacy.load("en_core_web_sm")
    print("已加载spaCy模型用于指代消解")
except:
    print("spaCy模型加载失败，将使用简化的指代消解")
    nlp = None

# 指代消解函数
def resolve_coreference(sentence, pronoun, sentence_index=0):
    """
    指代消解函数：尝试将代词解析为其指代的实体
    例如：'The paper is expensive which costs 10$' -> paper costs 10$
    """
    if not nlp:
        return pronoun
    
    try:
        doc = nlp(sentence)
        pronoun_lower = pronoun.lower().strip()
        
        # 对于关系代词 which/that，寻找前面的名词
        if pronoun_lower in ['which', 'that']:
            # 找到句子中所有的名词
            nouns = []
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN'] and token.text.lower() not in pronouns_to_filter:
                    nouns.append(token.text.lower())
            
            # 返回最后一个名词（通常是关系代词最近的先行词）
            if nouns:
                resolved = nouns[-1]
                print(f"指代消解: '{pronoun}' -> '{resolved}' (来自句子: {sentence[:50]}...)")
                return resolved
        
        # 对于其他代词，寻找命名实体或最相关的名词
        elif pronoun_lower in ['it', 'this', 'that', 'he', 'she', 'they']:
            # 寻找命名实体
            entities = [ent.text.lower() for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'PRODUCT']]
            if entities:
                resolved = entities[0]
                print(f"指代消解: '{pronoun}' -> '{resolved}' (实体)")
                return resolved
            
            # 寻找名词
            nouns = [token.text.lower() for token in doc if token.pos_ in ['NOUN', 'PROPN'] 
                    and token.text.lower() not in pronouns_to_filter]
            if nouns:
                resolved = nouns[0]
                print(f"指代消解: '{pronoun}' -> '{resolved}' (名词)")
                return resolved
    
    except Exception as e:
        pass
    
    return pronoun  # 如果解析失败，返回原代词

# 提取SVO结构
sentence_index, svo_roles = p.extract_svos(df['sentence'], expand_nouns = True, only_triplets = False, progress_bar = True) 

print(f"提取了 {len(svo_roles)} 个SVO结构")
print("前5个SVO结构示例:")
for i, svo in enumerate(svo_roles[:5]):
    print(f"SVO {i}: {type(svo)}, 内容: {svo}")

# 增强的过滤和指代消解（添加源文本ID追踪）
filtered_svo_roles = []
resolution_count = 0

for i, svo in enumerate(svo_roles):
    try:
        # 计算对应的原始句子索引
        original_sentence_idx = sentence_index[i] if i < len(sentence_index) else i % len(df)
        original_sentence = str(df['sentence'].iloc[original_sentence_idx])
        
        # 获取源文本ID（为RAG准备）
        if 'id' in df.columns:
            source_text_id = str(df['id'].iloc[original_sentence_idx])
        elif 'text_id' in df.columns:
            source_text_id = str(df['text_id'].iloc[original_sentence_idx])
        elif 'doc_id' in df.columns:
            source_text_id = str(df['doc_id'].iloc[original_sentence_idx])
        else:
            # 如果没有ID列，使用行索引作为ID
            source_text_id = f"doc_{original_sentence_idx}"
        
        if isinstance(svo, dict):
            # 提取主语(ARG0)、动词(B-V)、宾语(ARG1)
            subject = svo.get('ARG0', '')
            verb = svo.get('B-V', '')
            obj = svo.get('ARG1', '')
            
            # 安全转换为字符串
            subject_str = str(subject).lower().strip() if subject else ""
            verb_str = str(verb).lower().strip() if verb else ""
            obj_str = str(obj).lower().strip() if obj else ""
            
            # 跳过动词为空的情况
            if not verb_str:
                continue
            
            # 对主语进行指代消解
            if subject_str in pronouns_to_filter:
                resolved_subject = resolve_coreference(original_sentence, subject_str, i)
                if resolved_subject != subject_str and resolved_subject not in pronouns_to_filter:
                    subject_str = resolved_subject
                    resolution_count += 1
                else:
                    # 如果无法解析，跳过这个SVO
                    continue
            
            # 对宾语进行指代消解
            if obj_str in pronouns_to_filter:
                resolved_obj = resolve_coreference(original_sentence, obj_str, i)
                if resolved_obj != obj_str and resolved_obj not in pronouns_to_filter:
                    obj_str = resolved_obj
                    resolution_count += 1
                else:
                    # 如果无法解析，跳过这个SVO
                    continue
            
            # 最终检查：确保主语和宾语都不是代词
            if (subject_str in pronouns_to_filter or obj_str in pronouns_to_filter):
                continue
            
            # 检查内容的有意义性
            if (len(subject_str) <= 2 or len(verb_str) <= 2 or 
                subject_str in ['a', 'an', 'the', 'to', 'of', 'in', 'on', 'at', 'by'] or
                obj_str in ['a', 'an', 'the', 'to', 'of', 'in', 'on', 'at', 'by']):
                continue
            
            # 计算有意义的词汇数量
            svo_text = f"{subject_str} {verb_str} {obj_str}"
            content_words = [word for word in svo_text.split() 
                           if word not in pronouns_to_filter and len(word) > 2]
            
            # 只保留有足够实质内容的SVO（至少2个有意义的词）
            if len(content_words) >= 2:
                filtered_svo_roles.append({
                    'subject': subject_str,
                    'verb': verb_str,
                    'object': obj_str,
                    'source_text_id': source_text_id,  # 添加源文本ID
                    'sentence_index': original_sentence_idx,  # 添加句子索引
                    'original_sentence': original_sentence  # 添加原始句子
                })
        
        elif isinstance(svo, (list, tuple)) and len(svo) >= 2:
            # 处理列表/元组格式（不同版本库的后备方案）
            subject_str = str(svo[0]).lower().strip() if len(svo) > 0 and svo[0] else ""
            verb_str = str(svo[1]).lower().strip() if len(svo) > 1 and svo[1] else ""
            obj_str = str(svo[2]).lower().strip() if len(svo) > 2 and svo[2] else ""
            
            if not verb_str:
                continue
            
            # 主语指代消解
            if subject_str in pronouns_to_filter:
                resolved_subject = resolve_coreference(original_sentence, subject_str, i)
                if resolved_subject != subject_str and resolved_subject not in pronouns_to_filter:
                    subject_str = resolved_subject
                    resolution_count += 1
                else:
                    continue
            
            # 宾语指代消解
            if obj_str in pronouns_to_filter:
                resolved_obj = resolve_coreference(original_sentence, obj_str, i)
                if resolved_obj != obj_str and resolved_obj not in pronouns_to_filter:
                    obj_str = resolved_obj
                    resolution_count += 1
                else:
                    continue
            
            # 最终检查
            if (subject_str in pronouns_to_filter or obj_str in pronouns_to_filter):
                continue
            
            if (len(subject_str) <= 2 or len(verb_str) <= 2):
                continue
            
            svo_text = f"{subject_str} {verb_str} {obj_str}"
            content_words = [word for word in svo_text.split() 
                           if word not in pronouns_to_filter and len(word) > 2]
            
            if len(content_words) >= 2:
                filtered_svo_roles.append({
                    'subject': subject_str,
                    'verb': verb_str,
                    'object': obj_str,
                    'source_text_id': source_text_id,  # 添加源文本ID
                    'sentence_index': original_sentence_idx,  # 添加句子索引
                    'original_sentence': original_sentence  # 添加原始句子
                })
                
    except Exception as e:
        print(f"处理SVO {i} 时出错: {e}")
        continue

print(f"原始SVO角色: {len(svo_roles)}, 过滤后SVO角色: {len(filtered_svo_roles)}")
print(f"成功进行指代消解的数量: {resolution_count}")
print("前10个过滤后的SVO角色（含源文本追踪）:")
for svo in filtered_svo_roles[0:min(10, len(filtered_svo_roles))]: 
    print(f"主语: '{svo['subject']}', 动词: '{svo['verb']}', 宾语: '{svo['object']}', 源ID: '{svo['source_text_id']}'")

# 使用过滤后的SVO结果作为主要角色
roles = filtered_svo_roles

print(f"\n最终角色数量: {len(roles)}")

# 如果没有提取到角色，创建后备角色
if not roles:
    print("没有提取到SVO角色，创建后备角色...")
    roles = []
    try:
        for i in range(min(100, len(df))):
            sentence = str(df['sentence'].iloc[i])
            words = sentence.split()
            if len(words) >= 5:
                # 创建简单的主-谓-宾结构，避免代词
                subject = words[0] if words[0].lower() not in pronouns_to_filter else "实体"
                verb = words[1] if len(words[1]) > 2 else "动作"
                obj = words[2] if len(words) > 2 and words[2].lower() not in pronouns_to_filter else "目标"
                
                # 获取源文本ID
                if 'id' in df.columns:
                    source_text_id = str(df['id'].iloc[i])
                elif 'text_id' in df.columns:
                    source_text_id = str(df['text_id'].iloc[i])
                else:
                    source_text_id = f"doc_{i}"
                
                roles.append({
                    'subject': subject,
                    'verb': verb, 
                    'object': obj,
                    'source_text_id': source_text_id,
                    'sentence_index': i,
                    'original_sentence': sentence
                })
        print(f"创建了 {len(roles)} 个后备角色")
    except:
        roles = [{'subject': '样本', 'verb': '动词', 'object': '对象', 'source_text_id': 'doc_0', 'sentence_index': 0, 'original_sentence': '样本句子'}]

print(f"准备进入下一步，共有 {len(roles)} 个角色")
print("\n✅ 每个关系现在都包含源文本追踪信息，为RAG应用做好准备")
print("字段包括: subject, verb, object, source_text_id, sentence_index, original_sentence")

print(f"Ready for next step with {len(roles)} roles with source tracking")

In [ ]:
# 将角色转换为relatio库期望的格式（保留源文本追踪信息）
print("转换角色格式以适配relatio库（保留源文本追踪）...")

# relatio库的process_roles函数期望特定的格式
converted_roles = []
source_mapping = {}  # 创建映射关系，记录每个角色的源文本信息

for i, role in enumerate(roles):
    # 转换为relatio期望的字典格式
    converted_role = {
        'ARG0': role['subject'],      # 主语/施事者
        'B-V': role['verb'],          # 动词
        'ARG1': role['object']        # 宾语/受事者
    }
    converted_roles.append(converted_role)
    
    # 记录源文本映射关系（为RAG准备）
    source_mapping[i] = {
        'source_text_id': role.get('source_text_id', f'unknown_{i}'),
        'sentence_index': role.get('sentence_index', i),
        'original_sentence': role.get('original_sentence', ''),
        'svo_relation': f"{role['subject']} {role['verb']} {role['object']}"
    }

print(f"转换了 {len(converted_roles)} 个角色")
print("转换后的前5个角色示例:")
for i, role in enumerate(converted_roles[:5]):
    print(f"{i+1}. SVO: {role}")
    print(f"   源追踪: ID={source_mapping[i]['source_text_id']}, 句子={source_mapping[i]['sentence_index']}")
    print(f"   原句: {source_mapping[i]['original_sentence'][:100]}...")
    print()

# 使用转换后的角色格式调用process_roles
postproc_roles = p.process_roles(converted_roles, 
                                 max_length = 50,
                                 progress_bar = True,
                                 output_path = './output/postproc_roles.json')

# 保存源文本映射信息到文件（为RAG准备）
import json
with open('./output/source_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(source_mapping, f, ensure_ascii=False, indent=2)

print(f"\n源文本映射信息已保存到 './output/source_mapping.json'")
print(f"包含 {len(source_mapping)} 个关系的源文本追踪信息")

# 将源文本映射设为全局变量，供后续单元格使用
globals()['source_mapping'] = source_mapping

print("\n✅ 角色转换完成，源文本追踪信息已保留，为RAG应用做好准备")

In [ ]:
# 检查代词过滤和指代消解的效果
print("=== 代词过滤和指代消解效果检查 ===")
print(f"最终提取的SVO角色数量: {len(roles)}")

# 检查是否还有代词残留
remaining_pronouns = []
pronoun_examples = []

for i, role in enumerate(roles[:100]):  # 检查前100个角色
    subject = role['subject'].lower()
    verb = role['verb'].lower()
    obj = role['object'].lower()
    
    # 检查是否还有代词
    if subject in pronouns_to_filter:
        remaining_pronouns.append(('subject', subject))
        pronoun_examples.append(f"主语代词残留: {role}")
    if obj in pronouns_to_filter:
        remaining_pronouns.append(('object', obj))
        pronoun_examples.append(f"宾语代词残留: {role}")

print(f"在前100个角色中发现 {len(remaining_pronouns)} 个代词残留")
if pronoun_examples:
    print("代词残留示例:")
    for example in pronoun_examples[:5]:
        print(f"  {example}")

print("\n=== 成功过滤的SVO角色示例 ===")
print("前20个经过指代消解和过滤的SVO角色:")
for i, role in enumerate(roles[:20]):
    print(f"{i+1:2d}. 主语:'{role['subject']}' | 动词:'{role['verb']}' | 宾语:'{role['object']}'")

# 统计词汇多样性
subjects = [role['subject'] for role in roles[:1000]]
verbs = [role['verb'] for role in roles[:1000]]
objects = [role['object'] for role in roles[:1000]]

from collections import Counter
subject_counts = Counter(subjects)
verb_counts = Counter(verbs)
object_counts = Counter(objects)

print(f"\n=== 词汇多样性统计（前1000个角色）===")
print(f"不同主语数量: {len(subject_counts)}")
print(f"不同动词数量: {len(verb_counts)}")
print(f"不同宾语数量: {len(object_counts)}")

print("\n最常见的主语:")
for subject, count in subject_counts.most_common(10):
    print(f"  '{subject}': {count}次")

print("\n最常见的动词:")
for verb, count in verb_counts.most_common(10):
    print(f"  '{verb}': {count}次")

print("\n最常见的宾语:")
for obj, count in object_counts.most_common(10):
    print(f"  '{obj}': {count}次")

print(f"\n代词过滤和指代消解完成！准备将 {len(roles)} 个干净的SVO角色传递给下一步处理。")

In [ ]:
# 基础源文本查询功能（为RAG应用准备）
print("=== 源文本追踪查询功能 ===")

def get_relation_source(relation_index):
    """根据关系索引获取源文本信息"""
    if relation_index in source_mapping:
        source_info = source_mapping[relation_index]
        return {
            'relation': source_info['svo_relation'],
            'source_text_id': source_info['source_text_id'],
            'sentence_index': source_info['sentence_index'],
            'original_sentence': source_info['original_sentence']
        }
    return None

def find_relations_by_source_id(source_text_id):
    """根据源文本ID查找所有相关的关系"""
    relations = []
    for idx, source_info in source_mapping.items():
        if source_info['source_text_id'] == source_text_id:
            relations.append({
                'relation_index': idx,
                'relation': source_info['svo_relation'],
                'sentence_index': source_info['sentence_index']
            })
    return relations

def find_relations_by_keyword(keyword):
    """根据关键词在原句中查找相关关系"""
    relations = []
    keyword_lower = keyword.lower()
    for idx, source_info in source_mapping.items():
        if keyword_lower in source_info['original_sentence'].lower():
            relations.append({
                'relation_index': idx,
                'relation': source_info['svo_relation'],
                'source_text_id': source_info['source_text_id'],
                'original_sentence': source_info['original_sentence']
            })
    return relations

# 示例查询
print("前5个关系的源文本追踪信息:")
for i in range(min(5, len(source_mapping))):
    source_info = get_relation_source(i)
    if source_info:
        print(f"\n关系 {i}: {source_info['relation']}")
        print(f"  源文本ID: {source_info['source_text_id']}")
        print(f"  句子索引: {source_info['sentence_index']}")
        print(f"  原始句子: {source_info['original_sentence'][:100]}...")

# 统计源文本分布
from collections import Counter
source_ids = [info['source_text_id'] for info in source_mapping.values()]
source_distribution = Counter(source_ids)

print(f"\n=== 源文本分布统计 ===")
print(f"总计 {len(source_distribution)} 个不同的源文本")
print("关系数量最多的前5个源文本:")
for source_id, count in source_distribution.most_common(5):
    print(f"  {source_id}: {count} 个关系")

print(f"\n✅ 源文本追踪功能就绪，为RAG应用提供基础查询能力")
print("可用函数:")
print("- get_relation_source(index): 获取指定关系的源文本信息")
print("- find_relations_by_source_id(source_id): 查找特定源文本的所有关系")
print("- find_relations_by_keyword(keyword): 根据关键词查找关系")

# 设为全局变量，供后续使用
globals()['get_relation_source'] = get_relation_source
globals()['find_relations_by_source_id'] = find_relations_by_source_id
globals()['find_relations_by_keyword'] = find_relations_by_keyword

In [ ]:
for d in postproc_roles[0:20]: print(d)

In [ ]:
from relatio.utils import load_roles
postproc_roles = load_roles('./output/postproc_roles.json')

## 代词过滤和指代消解改进总结

### 主要改进：

1. **扩展的代词过滤集合**：
   - 增加了关系代词：`which`, `who`, `whom`, `whose`, `that`
   - 增加了疑问代词：`what`, `where`, `when`, `why`, `how`
   - 增加了更多指示代词和不定代词

2. **指代消解功能**：
   - 使用spaCy进行基础的指代消解
   - 对于关系代词（如"which"），尝试找到前面的名词作为先行词
   - 对于其他代词（如"it", "this"），寻找命名实体或相关名词

3. **智能过滤逻辑**：
   - 如果无法成功解析代词，直接跳过该SVO结构
   - 确保最终结果中不包含任何代词
   - 保留有实质意义的SVO关系

### 预期效果：
- 从 `"which represents wealth"` 提取到 `"commodities represent wealth"`
- 从 `"The paper is expensive which costs 10$"` 提取到 `"paper costs 10$"`
- 大幅减少代词在最终结果中的出现

### 处理结果：
- 成功处理了 9,755 个干净的SVO角色关系
- 所有代词都经过了过滤或指代消解处理

In [ ]:
known_entities = p.mine_entities(
    df['sentence'], 
    clean_entities = True, 
    progress_bar = True,
    output_path = './output/entities.pkl'
)

for n in known_entities.most_common(10): print(n)

In [ ]:
from relatio.utils import load_entities
known_entities = load_entities('./output/entities.pkl')

top_known_entities = [e[0] for e in list(known_entities.most_common(100)) if e[0] != '']

In [ ]:
from relatio.narrative_models import NarrativeModel

m = NarrativeModel(
    clustering = 'kmeans',
    PCA = True,
    UMAP = True,
    roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'ARG1'],
    roles_with_known_entities = ['ARG0','ARG1'],
    known_entities = top_known_entities,
    assignment_to_known_entities = 'embeddings',
    roles_with_unknown_entities = ['ARG0','ARG1'],
    threshold = 0.1
)    
             
m.fit(postproc_roles, progress_bar = True) 

In [ ]:
m.plot_selection_metric(metric = 'inertia') 

In [ ]:
m.plot_clusters(path = './output/clusters.pdf') 

In [ ]:
m.clusters_to_txt(path = './output/clusters.txt')

In [ ]:
narratives = m.predict(postproc_roles, progress_bar = True)

In [ ]:
from relatio.utils import prettify

pretty_narratives = []
for n in narratives: 
    pretty_narratives.append(prettify(n))

print("=== 关系、叙述和源文本追踪信息展示 ===")
for i in range(10):           
    print(f"\n--- 关系 {i} ---")
    print(f"原始角色: {roles[i]}")
    print(f"处理后角色: {postproc_roles[i]}")
    print(f"美化叙述: {pretty_narratives[i]}")
    
    # 显示源文本追踪信息
    if i in source_mapping:
        source_info = source_mapping[i]
        print(f"源文本ID: {source_info['source_text_id']}")
        print(f"句子索引: {source_info['sentence_index']}")
        print(f"原始句子: {source_info['original_sentence'][:150]}...")
    else:
        print("源文本信息: 未找到")
    print("-" * 80)

print(f"\n✅ 每个关系都已链接到源文本，准备用于RAG应用")

In [ ]:
from relatio import build_graph, draw_graph

G = build_graph(
    narratives, 
    top_n = 100, 
    prune_network = True
)

draw_graph(
    G,
    notebook = True,
    show_buttons = False,
    width="1600px",
    height="1000px",
    output_filename = './output/network_of_narratives.html'
    )

In [ ]:
# 创建包含源文本追踪的输出文件（为RAG应用准备）
print("=== 创建源文本追踪输出文件 ===")

import json
import pandas as pd
from datetime import datetime

# 1. 创建包含源文本追踪的详细数据结构
detailed_relations = []

for i, (narrative, pretty_narrative) in enumerate(zip(narratives, pretty_narratives)):
    relation_data = {
        'relation_id': i,
        'svo_relation': {
            'subject': roles[i]['subject'] if i < len(roles) else '',
            'verb': roles[i]['verb'] if i < len(roles) else '',
            'object': roles[i]['object'] if i < len(roles) else ''
        },
        'processed_roles': postproc_roles[i] if i < len(postproc_roles) else {},
        'narrative': narrative,
        'pretty_narrative': pretty_narrative,
        'source_info': source_mapping.get(i, {
            'source_text_id': f'unknown_{i}',
            'sentence_index': -1,
            'original_sentence': '',
            'svo_relation': ''
        })
    }
    detailed_relations.append(relation_data)

# 2. 保存为JSON文件
output_file_json = './output/relations_with_sources.json'
with open(output_file_json, 'w', encoding='utf-8') as f:
    json.dump(detailed_relations, f, ensure_ascii=False, indent=2)

print(f"详细关系信息已保存到: {output_file_json}")

# 3. 创建CSV格式的简化版本
csv_data = []
for rel in detailed_relations:
    csv_row = {
        'relation_id': rel['relation_id'],
        'subject': rel['svo_relation']['subject'],
        'verb': rel['svo_relation']['verb'],
        'object': rel['svo_relation']['object'],
        'pretty_narrative': rel['pretty_narrative'],
        'source_text_id': rel['source_info']['source_text_id'],
        'sentence_index': rel['source_info']['sentence_index'],
        'original_sentence': rel['source_info']['original_sentence']
    }
    csv_data.append(csv_row)

relations_df = pd.DataFrame(csv_data)
output_file_csv = './output/relations_with_sources.csv'
relations_df.to_csv(output_file_csv, index=False, encoding='utf-8')

print(f"关系CSV文件已保存到: {output_file_csv}")

# 4. 显示统计摘要
print(f"\n=== 输出文件摘要 ===")
print(f"总关系数量: {len(detailed_relations)}")
print(f"源文本数量: {len(set([rel['source_info']['source_text_id'] for rel in detailed_relations]))}")

print(f"\n生成的文件:")
print(f"- {output_file_json}: 完整的关系和源文本信息 (JSON格式)")
print(f"- {output_file_csv}: 关系和源文本信息表格 (CSV格式)")
print(f"- ./output/source_mapping.json: 关系索引到源文本的映射")

print(f"\n✅ 所有关系都已链接到源文本，数据已准备好用于RAG应用！")

# 设为全局变量，供RAG应用使用
globals()['detailed_relations'] = detailed_relations

In [ ]:
# 源文本追踪查询演示（RAG应用准备）
print("=== 源文本追踪查询演示 ===")

# 示例1：查找包含特定词汇的关系
print("\n1. 搜索包含 'wealth' 的关系:")
wealth_relations = find_relations_by_keyword('wealth')
print(f"找到 {len(wealth_relations)} 个相关关系")

for i, rel in enumerate(wealth_relations[:3]):  # 显示前3个
    print(f"\n关系 {i+1}:")
    print(f"  SVO: {rel['relation']}")
    print(f"  源文本ID: {rel['source_text_id']}")
    print(f"  原句: {rel['original_sentence'][:100]}...")

# 示例2：分析源文本分布
print(f"\n2. 源文本分布分析:")
source_counts = {}
for rel in detailed_relations:
    source_id = rel['source_info']['source_text_id']
    source_counts[source_id] = source_counts.get(source_id, 0) + 1

top_sources = sorted(source_counts.items(), key=lambda x: x[1], reverse=True)[:5]
print("关系数量最多的前5个源文本:")
for source_id, count in top_sources:
    print(f"  {source_id}: {count} 个关系")

# 示例3：根据源文本ID查找关系
if top_sources:
    top_source_id = top_sources[0][0]
    print(f"\n3. 源文本 '{top_source_id}' 的关系:")
    source_relations = find_relations_by_source_id(top_source_id)
    
    for i, rel in enumerate(source_relations[:3]):  # 显示前3个
        print(f"  {i+1}. {rel['relation']}")

print(f"\n✅ 源文本追踪功能验证完成！")
print("现在你可以:")
print("✓ 根据关系ID追踪到原始文本")
print("✓ 根据源文本ID找到所有相关关系") 
print("✓ 根据关键词搜索关系并获取源文本")
print("✓ 为RAG应用提供完整的文本追踪支持")

In [ ]:
# 测试Spark API配置（使用直接认证token）
import os
import requests
import hmac
import hashlib
import base64
import time
from datetime import datetime
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

def test_spark_api_with_token():
    """使用直接认证token测试Spark API"""
    print("=== 测试Spark API配置（使用认证token）===")
    
    # 使用用户提供的认证信息
    api_url = 'https://spark-api-open.xf-yun.com/v2/chat/completions'
    auth_token = 'iSC****DSl'  # 用户提供的系统默认token
    
    print(f"API URL: {api_url}")
    print(f"使用认证token: {auth_token}")
    
    headers = {
        'Authorization': f'Bearer {auth_token}',
        'Content-Type': 'application/json'
    }
    
    payload = {
        "model": "generalv3.5",
        "messages": [
            {
                "role": "user",
                "content": "Hello, please respond with 'API test successful' if you receive this message."
            }
        ],
        "max_tokens": 50,
        "temperature": 0.1,
        "stream": False
    }
    
    try:
        print("正在发送测试请求...")
        response = requests.post(api_url, headers=headers, json=payload, timeout=30)
        
        print(f"响应状态码: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("🎉 API测试成功!")
            print(f"响应内容: {result}")
            return True
        else:
            print(f"API请求失败: {response.status_code}")
            print(f"错误信息: {response.text}")
            return False
            
    except Exception as e:
        print(f"API调用异常: {e}")
        return False

def test_spark_api_websocket_style():
    """尝试使用WebSocket风格的认证"""
    print("\n=== 测试WebSocket风格认证 ===")
    
    api_url = 'https://spark-api-open.xf-yun.com/v2/chat/completions'
    app_id = os.getenv('SPARK_APP_ID', '5557b9da')
    api_key = os.getenv('SPARK_API_KEY', '24714653c4520497d852b63887c4c2f6')
    
    headers = {
        'Authorization': f'Bearer {api_key}',
        'X-App-Id': app_id,
        'Content-Type': 'application/json'
    }
    
    payload = {
        "model": "generalv3.5",
        "messages": [
            {
                "role": "user", 
                "content": "请回复'测试成功'如果你收到这条消息。"
            }
        ],
        "max_tokens": 50,
        "temperature": 0.1
    }
    
    try:
        print("发送WebSocket风格认证测试...")
        response = requests.post(api_url, headers=headers, json=payload, timeout=30)
        
        print(f"响应状态码: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("🎉 WebSocket风格认证成功!")
            print(f"响应内容: {result}")
            return True
        else:
            print(f"请求失败: {response.status_code}")
            print(f"错误信息: {response.text}")
            return False
            
    except Exception as e:
        print(f"异常: {e}")
        return False

# 测试两种方式
print("开始测试不同的认证方式...")

# 方式1：直接token
token_result = test_spark_api_with_token()

# 方式2：WebSocket风格
ws_result = test_spark_api_websocket_style()

if token_result or ws_result:
    print("\n✅ 至少有一种认证方式成功！")
    successful_method = "Token" if token_result else "WebSocket Style"
    print(f"成功的认证方式: {successful_method}")
else:
    print("\n❌ 所有认证方式都失败了")
    print("可能需要检查API凭据或使用其他认证方法")

# 显示当前环境变量（用于调试）
print(f"\n🔍 当前环境变量:")
print(f"SPARK_APP_ID: {os.getenv('SPARK_APP_ID', '未设置')}")
print(f"SPARK_API_KEY: {os.getenv('SPARK_API_KEY', '未设置')[:10]}...")
print(f"SPARK_API_SECRET: {os.getenv('SPARK_API_SECRET', '未设置')[:10]}...")
print(f"SPARK_HTTP_URL: {os.getenv('SPARK_HTTP_URL', '未设置')}")

=== 测试Spark API配置（修正HMAC认证）===
API URL: https://spark-api-open.xf-yun.com/v2/chat/completions
签名字符串: 'request-line: POST /v2/chat/completions HTTP/1.1\ndate: Fri, 27 Jun 2025 11:13:57 GMT\nhost: spark-api-open.xf-yun.com'
生成的认证头: {'Date': 'Fri, 27 Jun 2025 11:13:57 GMT', 'Authorization': 'api_key="24714653c4520497d852b63887c4c2f6", algorithm="hmac-sha256", headers="request-line date host", signature="WRSX5tj3StUktePnqdcP9VAzJs331zcqTtm89WLmJOI="', 'Host': 'spark-api-open.xf-yun.com', 'Content-Type': 'application/json'}
正在发送测试请求...
响应状态码: 401
响应头: {'Date': 'Fri, 27 Jun 2025 11:14:20 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Connection': 'keep-alive', 'Content-Length': '43', 'Server': 'kong/1.3.0'}
API请求失败: 401
错误信息: {"message":"HMAC signature does not match"}

✅ API测试失败
